In [1]:
pip install implicit==0.4.8

     |████████████████████████████████| 1.1 MB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for implicit: filename=implicit-0.4.8-cp37-cp37m-linux_x86_64.whl size=4606535 sha256=f188b254e9b4e4f9a8ed19c3fc184aee33ef33e73a028b9ea21053d2e26d1dde
  Stored in directory: /root/.cache/pip/wheels/88/e6/34/25e73cccbaf1a961154bb562a5f86123b68fdbf40e306073d6
Successfully built implicit


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
root = '/content/drive/My Drive/rec_sys/lesson2/'

In [5]:
data = pd.read_csv(root + '/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [6]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [7]:
data = data_train['item_id'].unique()
data

array([ 1004906,  1033142,  1036325, ..., 15722756, 17170636, 15716393])

In [8]:
df_2 = pd.DataFrame(data, columns=['item_id'])

In [9]:
weights = data_train.groupby(['item_id'])['sales_value'].sum()

In [10]:
weigt_id = pd.DataFrame(weights, columns=['sales_value'])

In [11]:
item_weights = pd.merge(df_2, weigt_id, on='item_id')

In [12]:
item_weights.head(5)

,item_id,sales_value
0,1004906,8232.66
1,1033142,2210.90
2,1036325,284.41
3,1082185,26029.96
4,8160430,296.94


Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж

    Можно сэмплировать товары случайно, но пропорционально какому-либо весу
    Например, прямопропорционально популярности. Вес = log(sales_sum товара)


In [13]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items_weights = np.array(items_weights)
    recs = np.random.choice(np.log(items_weights[:,1]), size = n, replace=False)
    return recs.tolist()

In [14]:
items_weight_array = np.array(item_weights)

In [15]:
print(items_weight_array[:,:])

[[1.0049060e+06 8.2326600e+03]
 [1.0331420e+06 2.2109000e+03]
 [1.0363250e+06 2.8441000e+02]
 ...
 [1.5722756e+07 1.0000000e+00]
 [1.7170636e+07 2.5000000e+01]
 [1.5716393e+07 7.7700000e+00]]


In [16]:
%%time

# your_code
print(weighted_random_recommendation(item_weights,n=5))

[1.4085449700547104, 1.2499017362143359, 0.8285518175661483, 2.194999882314108, 2.6658383522929006]
CPU times: user 11.2 ms, sys: 1.85 ms, total: 13.1 ms
Wall time: 15.1 ms


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log
  del sys.path[0]


Задание 2. Расчет метрик

Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [18]:
result = pd.read_csv(root + '/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[12605426, 1220599, 6391133, 9836719, 28015]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[88350, 7025500, 9677145, 7442949, 943671]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [47]:
# your_code
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [ ]:
result.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual']), axis=1).mean()



0.0004897159647404506

In [ ]:
result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual'], k=5), axis=1).mean()

0.15523996082272082

In [ ]:
result.apply(lambda row: precision_at_k(row['itemitem'], row['actual']), axis=1).mean()

0.13692458374142857

In [ ]:
result.apply(lambda row: precision_at_k(row['cosine'], row['actual']), axis=1).mean()

0.13290891283055686

In [ ]:
result.apply(lambda row: precision_at_k(row['tfidf'], row['actual']), axis=1).mean()

0.16092066601371002

### Наилучшие результаты показал алгоритм "tfidf"